In [1]:
cd ..

/Users/muhammadfarooq/Codebase/thesis-experiment/Pith2Bark
/Users/muhammadfarooq/Codebase/thesis-experiment/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

# Damage Dataset

In [3]:
def read_damage_min_max(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_mean']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_min_max': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [4]:
def read_damage_mean(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_min_max']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_mean': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [5]:
#damage
damage_sixteen_url = "damage/damage_sixteen.csv"
damage_thirtytwo_url = "damage/damage_thirtytwo.csv"

df_damage_sixteen_min_max = read_damage_min_max(damage_sixteen_url)
df_damage_thirtytwo_min_max = read_damage_min_max(damage_thirtytwo_url)

df_damage_sixteen_mean = read_damage_mean(damage_sixteen_url)
df_damage_thirtytwo_mean = read_damage_mean(damage_thirtytwo_url)

In [6]:
#df_damage_sixteen_min_max

In [7]:
#df_damage_thirtytwo_min_max

In [8]:
#df_damage_sixteen_mean

In [9]:
#df_damage_thirtytwo_mean

In [10]:
#STOP

In [11]:
df = df_damage_sixteen_min_max

In [12]:
#df

In [13]:
# set up the file paths for your image data and labels
img_size = (128, 128)

data_dir = 'damage/images'

# convert the DataFrame to a dictionary with specific column names as key and value
label_dict = {k: v for k, v in zip(df['image'], df['Damage'])}

# create empty lists to store the image data and labels
images = []
labels = []

# loop through each image file in the directory
for filename in os.listdir(data_dir):
    # load the image file and convert it to a NumPy array
    image = load_img(os.path.join(data_dir, filename), target_size=img_size)
    image = img_to_array(image)
    # preprocess the image by scaling the pixel values to be between 0 and 1
    image = image.astype('float32') / 255.0
    # add the preprocessed image and its label to the lists
    images.append(image)
    labels.append(label_dict[filename])

# convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# print the shape of the data to verify that it's been loaded correctly
print('Image data shape:', images.shape) #(148, 64, 64, 3)
print('Label data shape:', labels.shape) #(148,)

Image data shape: (148, 128, 128, 3)
Label data shape: (148,)
Image data shape: (148, 128, 128, 3)
Label data shape: (148,)


In [14]:
#STOP

# Config

In [15]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [16]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [17]:
units = [64]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

In [18]:
#STOP

# Model

In [19]:
class Baseline():
    def __init__(self, _input_dim, units, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # convolutional layer
        model.add(Conv2D(units, kernel_size=kernel_size, activation=activation, input_shape=_input_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
                  
        # flatten output of conv
        model.add(Flatten())
                  
        # hidden layer
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout))
                  
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [20]:
X = images
y = labels

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [21]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [22]:
input_dim = X_train.shape[1:]

In [23]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
                            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
                            
                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 64 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1

Units: 64 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
Epoch 1/50
3/3 - 3s - loss: 3.7785 - accuracy: 0.4886 - val_loss: 2.7688 - val_accuracy: 0.3556 - 3s/epoch - 911ms/step
3/3 - 3s - loss: 3.7785 - accuracy: 0.4886 - val_loss: 2.7688 - val_accuracy: 0.3556 - 3s/epoch - 911ms/step
Epoch 2/50
Epoch 2/50
3/3 - 1s - loss: 1.6068 - accuracy: 0.6932 - val_loss: 1.8359 - val_accuracy: 0.8667 - 1s/epoch - 373ms/step
3/3 - 1s - loss: 1.6068 - accuracy: 0.6932 - val_loss: 1.8359 - val_accuracy: 0.8667 - 1s/epoch - 373ms/step
Epoch 3/50
Epoch 3/50
3/3 - 1s - loss: 1.6393 - accuracy: 0.8523 - val_loss: 1.4651 - val_accuracy: 0.8000 - 1s/epoch - 381ms/step
3/3 - 1s - loss: 1.6393 - accuracy: 0.8523 - val_loss: 1.4651 - val_accuracy: 0.8000 - 1s/epoch - 381ms/step
Epoch 4/50
Epoch 4/50
3/3 - 1s - loss: 1.0608 - accuracy: 0.8409 - val_loss: 1.0372 - val_accu

3/3 - 1s - loss: 0.0767 - accuracy: 0.9888 - val_loss: 0.7035 - val_accuracy: 0.8182 - 1s/epoch - 360ms/step
Epoch 14/50
Epoch 14/50
3/3 - 1s - loss: 0.0683 - accuracy: 1.0000 - val_loss: 0.7564 - val_accuracy: 0.7955 - 1s/epoch - 354ms/step
3/3 - 1s - loss: 0.0683 - accuracy: 1.0000 - val_loss: 0.7564 - val_accuracy: 0.7955 - 1s/epoch - 354ms/step
Epoch 15/50
Epoch 15/50
3/3 - 1s - loss: 0.0564 - accuracy: 0.9888 - val_loss: 0.8068 - val_accuracy: 0.7955 - 1s/epoch - 358ms/step
3/3 - 1s - loss: 0.0564 - accuracy: 0.9888 - val_loss: 0.8068 - val_accuracy: 0.7955 - 1s/epoch - 358ms/step
Epoch 16/50
Epoch 16/50
3/3 - 1s - loss: 0.0427 - accuracy: 1.0000 - val_loss: 0.8271 - val_accuracy: 0.8182 - 1s/epoch - 362ms/step
3/3 - 1s - loss: 0.0427 - accuracy: 1.0000 - val_loss: 0.8271 - val_accuracy: 0.8182 - 1s/epoch - 362ms/step
Epoch 17/50
Epoch 17/50
3/3 - 1s - loss: 0.0309 - accuracy: 1.0000 - val_loss: 0.9184 - val_accuracy: 0.8182 - 1s/epoch - 364ms/step
3/3 - 1s - loss: 0.0309 - accura

Epoch 13/50
Epoch 13/50
3/3 - 1s - loss: 0.1131 - accuracy: 0.9659 - val_loss: 0.5397 - val_accuracy: 0.7556 - 1s/epoch - 344ms/step
3/3 - 1s - loss: 0.1131 - accuracy: 0.9659 - val_loss: 0.5397 - val_accuracy: 0.7556 - 1s/epoch - 344ms/step
Epoch 14/50
Epoch 14/50
3/3 - 1s - loss: 0.1100 - accuracy: 0.9432 - val_loss: 0.4695 - val_accuracy: 0.7778 - 1s/epoch - 343ms/step
3/3 - 1s - loss: 0.1100 - accuracy: 0.9432 - val_loss: 0.4695 - val_accuracy: 0.7778 - 1s/epoch - 343ms/step
Epoch 15/50
Epoch 15/50
3/3 - 1s - loss: 0.0830 - accuracy: 1.0000 - val_loss: 0.3320 - val_accuracy: 0.8667 - 1s/epoch - 365ms/step
3/3 - 1s - loss: 0.0830 - accuracy: 1.0000 - val_loss: 0.3320 - val_accuracy: 0.8667 - 1s/epoch - 365ms/step
Epoch 16/50
Epoch 16/50
3/3 - 1s - loss: 0.0774 - accuracy: 0.9773 - val_loss: 0.3723 - val_accuracy: 0.8667 - 1s/epoch - 354ms/step
3/3 - 1s - loss: 0.0774 - accuracy: 0.9773 - val_loss: 0.3723 - val_accuracy: 0.8667 - 1s/epoch - 354ms/step
Epoch 17/50
Epoch 17/50
3/3 - 1s

Epoch 1/50
Epoch 1/50
3/3 - 2s - loss: 2.7119 - accuracy: 0.6404 - val_loss: 1.0390 - val_accuracy: 0.9091 - 2s/epoch - 659ms/step
3/3 - 2s - loss: 2.7119 - accuracy: 0.6404 - val_loss: 1.0390 - val_accuracy: 0.9091 - 2s/epoch - 659ms/step
Epoch 2/50
Epoch 2/50
3/3 - 1s - loss: 1.5086 - accuracy: 0.7978 - val_loss: 1.0930 - val_accuracy: 0.7955 - 1s/epoch - 367ms/step
3/3 - 1s - loss: 1.5086 - accuracy: 0.7978 - val_loss: 1.0930 - val_accuracy: 0.7955 - 1s/epoch - 367ms/step
Epoch 3/50
Epoch 3/50
3/3 - 1s - loss: 1.8741 - accuracy: 0.6180 - val_loss: 0.3634 - val_accuracy: 0.9091 - 1s/epoch - 368ms/step
3/3 - 1s - loss: 1.8741 - accuracy: 0.6180 - val_loss: 0.3634 - val_accuracy: 0.9091 - 1s/epoch - 368ms/step
Epoch 4/50
Epoch 4/50
3/3 - 1s - loss: 1.1729 - accuracy: 0.7978 - val_loss: 0.5402 - val_accuracy: 0.9091 - 1s/epoch - 380ms/step
3/3 - 1s - loss: 1.1729 - accuracy: 0.7978 - val_loss: 0.5402 - val_accuracy: 0.9091 - 1s/epoch - 380ms/step
Epoch 5/50
Epoch 5/50
3/3 - 1s - loss: 0

3/3 - 1s - loss: 0.1198 - accuracy: 0.9318 - val_loss: 0.4562 - val_accuracy: 0.8000 - 1s/epoch - 378ms/step
Epoch 18/50
Epoch 18/50
3/3 - 1s - loss: 0.1003 - accuracy: 0.9773 - val_loss: 0.4719 - val_accuracy: 0.8000 - 1s/epoch - 377ms/step
3/3 - 1s - loss: 0.1003 - accuracy: 0.9773 - val_loss: 0.4719 - val_accuracy: 0.8000 - 1s/epoch - 377ms/step
Epoch 19/50
Epoch 19/50
3/3 - 1s - loss: 0.0906 - accuracy: 0.9545 - val_loss: 0.4033 - val_accuracy: 0.8667 - 1s/epoch - 358ms/step
3/3 - 1s - loss: 0.0906 - accuracy: 0.9545 - val_loss: 0.4033 - val_accuracy: 0.8667 - 1s/epoch - 358ms/step
Epoch 20/50
Epoch 20/50
3/3 - 1s - loss: 0.0851 - accuracy: 0.9659 - val_loss: 0.3597 - val_accuracy: 0.8667 - 1s/epoch - 379ms/step
3/3 - 1s - loss: 0.0851 - accuracy: 0.9659 - val_loss: 0.3597 - val_accuracy: 0.8667 - 1s/epoch - 379ms/step
Epoch 21/50
Epoch 21/50
3/3 - 1s - loss: 0.0731 - accuracy: 1.0000 - val_loss: 0.3781 - val_accuracy: 0.8667 - 1s/epoch - 345ms/step
3/3 - 1s - loss: 0.0731 - accura

...

3/3 - 1s - loss: 0.6972 - accuracy: 0.6023 - val_loss: 0.6495 - val_accuracy: 0.6444 - 1s/epoch - 435ms/step
Epoch 4/50
3/3 - 1s - loss: 0.6768 - accuracy: 0.6023 - val_loss: 0.6480 - val_accuracy: 0.6444 - 1s/epoch - 491ms/step
Epoch 5/50
3/3 - 1s - loss: 0.7000 - accuracy: 0.6250 - val_loss: 0.6496 - val_accuracy: 0.6444 - 1s/epoch - 409ms/step
Epoch 6/50
3/3 - 2s - loss: 0.6904 - accuracy: 0.5909 - val_loss: 0.6805 - val_accuracy: 0.6444 - 2s/epoch - 548ms/step
Epoch 7/50
3/3 - 1s - loss: 0.6997 - accuracy: 0.5568 - val_loss: 0.6575 - val_accuracy: 0.6444 - 1s/epoch - 489ms/step
Epoch 8/50
3/3 - 1s - loss: 0.7270 - accuracy: 0.5341 - val_loss: 0.6474 - val_accuracy: 0.6444 - 1s/epoch - 451ms/step
Epoch 9/50
3/3 - 1s - loss: 0.7152 - accuracy: 0.5568 - val_loss: 0.6467 - val_accuracy: 0.6444 - 1s/epoch - 445ms/step
Epoch 10/50
3/3 - 1s - loss: 0.6409 - accuracy: 0.6477 - val_loss: 0.6588 - val_accuracy: 0.6444 - 1s/epoch - 383ms/step
Epoch 11/50
3/3 - 1s - loss: 0.7338 - accuracy: 0.

Epoch 6/50
3/3 - 1s - loss: 0.7127 - accuracy: 0.5281 - val_loss: 0.6069 - val_accuracy: 0.7500 - 1s/epoch - 377ms/step
Epoch 7/50
3/3 - 1s - loss: 0.7269 - accuracy: 0.5393 - val_loss: 0.5962 - val_accuracy: 0.7500 - 1s/epoch - 373ms/step
Epoch 8/50
3/3 - 1s - loss: 0.7317 - accuracy: 0.5169 - val_loss: 0.6669 - val_accuracy: 0.7500 - 1s/epoch - 412ms/step
Epoch 9/50
3/3 - 1s - loss: 0.7011 - accuracy: 0.5169 - val_loss: 0.6483 - val_accuracy: 0.7500 - 1s/epoch - 426ms/step
Epoch 10/50
3/3 - 1s - loss: 0.7683 - accuracy: 0.4157 - val_loss: 0.6009 - val_accuracy: 0.7500 - 1s/epoch - 408ms/step
Epoch 11/50
3/3 - 1s - loss: 0.6978 - accuracy: 0.5618 - val_loss: 0.6265 - val_accuracy: 0.7500 - 1s/epoch - 390ms/step
Epoch 12/50
3/3 - 1s - loss: 0.6827 - accuracy: 0.5618 - val_loss: 0.6113 - val_accuracy: 0.7500 - 1s/epoch - 426ms/step
Epoch 13/50
3/3 - 1s - loss: 0.6972 - accuracy: 0.6067 - val_loss: 0.6413 - val_accuracy: 0.7500 - 1s/epoch - 425ms/step
Epoch 14/50
3/3 - 1s - loss: 0.7364 

3/3 - 1s - loss: 0.6317 - accuracy: 0.6966 - val_loss: 0.7529 - val_accuracy: 0.5000 - 1s/epoch - 398ms/step
Epoch 7/50
3/3 - 1s - loss: 0.6497 - accuracy: 0.6966 - val_loss: 0.7293 - val_accuracy: 0.5000 - 1s/epoch - 374ms/step
Epoch 8/50
3/3 - 1s - loss: 0.6317 - accuracy: 0.6966 - val_loss: 0.7610 - val_accuracy: 0.5000 - 1s/epoch - 377ms/step
Epoch 9/50
3/3 - 1s - loss: 0.6172 - accuracy: 0.7079 - val_loss: 0.8078 - val_accuracy: 0.5000 - 1s/epoch - 379ms/step
Epoch 10/50
3/3 - 1s - loss: 0.6148 - accuracy: 0.6854 - val_loss: 0.7946 - val_accuracy: 0.5000 - 1s/epoch - 394ms/step
Epoch 11/50
3/3 - 1s - loss: 0.6132 - accuracy: 0.6966 - val_loss: 0.7883 - val_accuracy: 0.5000 - 1s/epoch - 407ms/step
Epoch 12/50
3/3 - 1s - loss: 0.6098 - accuracy: 0.6966 - val_loss: 0.7398 - val_accuracy: 0.5000 - 1s/epoch - 368ms/step
Epoch 1/50
3/3 - 2s - loss: 0.7360 - accuracy: 0.5843 - val_loss: 0.5890 - val_accuracy: 0.7500 - 2s/epoch - 742ms/step
Epoch 2/50
3/3 - 1s - loss: 0.7121 - accuracy: 0

Epoch 50/50
3/3 - 1s - loss: 0.5598 - accuracy: 0.7614 - val_loss: 0.6066 - val_accuracy: 0.7556 - 1s/epoch - 459ms/step
Epoch 1/50
3/3 - 3s - loss: 0.6729 - accuracy: 0.5955 - val_loss: 1.0526 - val_accuracy: 0.5000 - 3s/epoch - 933ms/step
Epoch 2/50
3/3 - 1s - loss: 0.7440 - accuracy: 0.6629 - val_loss: 0.8869 - val_accuracy: 0.5000 - 1s/epoch - 430ms/step
Epoch 3/50
3/3 - 1s - loss: 0.6290 - accuracy: 0.6966 - val_loss: 0.8202 - val_accuracy: 0.5000 - 1s/epoch - 418ms/step
Epoch 4/50
3/3 - 1s - loss: 0.6288 - accuracy: 0.6854 - val_loss: 0.7614 - val_accuracy: 0.5000 - 1s/epoch - 422ms/step
Epoch 5/50
3/3 - 1s - loss: 0.6143 - accuracy: 0.6966 - val_loss: 0.7857 - val_accuracy: 0.5000 - 1s/epoch - 394ms/step
Epoch 6/50
3/3 - 1s - loss: 0.6307 - accuracy: 0.6629 - val_loss: 0.8155 - val_accuracy: 0.5000 - 1s/epoch - 397ms/step
Epoch 7/50
3/3 - 1s - loss: 0.6299 - accuracy: 0.6854 - val_loss: 0.7636 - val_accuracy: 0.5000 - 1s/epoch - 416ms/step
Epoch 8/50
3/3 - 1s - loss: 0.6409 - ac

Epoch 31/50
3/3 - 1s - loss: 0.6067 - accuracy: 0.6818 - val_loss: 0.6129 - val_accuracy: 0.6444 - 1s/epoch - 439ms/step
Epoch 32/50
3/3 - 1s - loss: 0.6395 - accuracy: 0.6136 - val_loss: 0.6186 - val_accuracy: 0.7333 - 1s/epoch - 409ms/step
Epoch 33/50
3/3 - 1s - loss: 0.6225 - accuracy: 0.6477 - val_loss: 0.6282 - val_accuracy: 0.7333 - 1s/epoch - 433ms/step
Epoch 34/50
3/3 - 1s - loss: 0.6093 - accuracy: 0.7159 - val_loss: 0.6070 - val_accuracy: 0.6444 - 1s/epoch - 433ms/step
Epoch 35/50
3/3 - 1s - loss: 0.6040 - accuracy: 0.6818 - val_loss: 0.6046 - val_accuracy: 0.6444 - 1s/epoch - 428ms/step
Epoch 36/50
3/3 - 1s - loss: 0.6193 - accuracy: 0.6364 - val_loss: 0.6015 - val_accuracy: 0.6444 - 1s/epoch - 435ms/step
Epoch 37/50
3/3 - 1s - loss: 0.6139 - accuracy: 0.6591 - val_loss: 0.6138 - val_accuracy: 0.7333 - 1s/epoch - 437ms/step
Epoch 38/50
3/3 - 1s - loss: 0.6002 - accuracy: 0.7159 - val_loss: 0.6032 - val_accuracy: 0.7556 - 1s/epoch - 430ms/step
Epoch 39/50
3/3 - 1s - loss: 0.5

In [24]:
print(best_acc)
print(baseline_values)

0.9090909361839294
{'units': 64, 'activation': 'relu', 'kernel_size': 3, 'learning_rate': 0.001, 'optimizer': 'Adam', 'dropout': 0.2}


In [25]:
#STOP